###Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

Q2

In [ ]:
data=pd.read_csv("/content/House Pricing.csv")

Load the dataset into a Pandas dataframe and display the first few rows.

In [ ]:
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Discretize the "age" variable into three bins: 'Young', 'Middle-aged', and 'Old'.

In [ ]:
hew=data['age'].describe()
hew

count    506.000000
mean      68.574901
std       28.148861
min        2.900000
25%       45.025000
50%       77.500000
75%       94.075000
max      100.000000
Name: age, dtype: float64

In [ ]:
min=hew[3]
max=hew[7]
print("MIN AGE" ,min)
print("MAX AGE" ,max)

MIN AGE 2.9
MAX AGE 100.0


In [ ]:

binningRanges = [min, min + (max - min) / 3, min + 2 * (max - min) / 3, max]

labels = ['Young', 'Middle-aged', 'Old']

data['groupedAge'] = pd.cut(data['age'], bins=binningRanges, labels=labels)

In [ ]:
print(data['groupedAge'])

0      Middle-aged
1              Old
2      Middle-aged
3      Middle-aged
4      Middle-aged
          ...     
501            Old
502            Old
503            Old
504            Old
505            Old
Name: groupedAge, Length: 506, dtype: category
Categories (3, object): ['Young' < 'Middle-aged' < 'Old']


Create a binary variable "is_charles_river" based on the "chas" column.

In [ ]:
data['charBinary'] = data['chas'].astype(bool)
data['charBinary']

0      False
1      False
2      False
3      False
4      False
       ...  
501    False
502    False
503    False
504    False
505    False
Name: charBinary, Length: 506, dtype: bool

In [ ]:
for index, row in data.iterrows():
    if row['charBinary']  == False:
        data.at[index, 'charBinary']  = 0
    else:
        data.at[index, 'charBinary']  = 1


In [ ]:
countvalues = data['charBinary'].value_counts()
print("Count of 0s:", countvalues[0])
print("Count of 1s:", countvalues[1])

Count of 0s: 471
Count of 1s: 35


In [ ]:

def outlierinterQuartleRange(data, threshold=1.5):

    newData = pd.DataFrame(columns=data.columns)


    for column in data.columns:
      if data[column].dtype in ['int64', 'float64']:
        quartileOner = data[column].quantile(0.25)
        quartileThree= data[column].quantile(0.75)
        interQuartleRange = quartileThree- quartileOner

        LB = quartileOner - threshold * interQuartleRange
        UB= quartileThree+ threshold * interQuartleRange

        outliers = data[(data[column] < LB) | (data[column] > UB)]
        notSoOutliers= data[(data[column] >= LB) & (data[column] <= UB)]
        newData = pd.concat([newData, notSoOutliers])

    return newData


In [ ]:
newData = outlierinterQuartleRange(data)

print(newData.head)


<bound method NDFrame.head of         crim    zn  indus chas    nox     rm   age     dis rad  tax  ptratio  \
0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296     15.3   
1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242     17.8   
2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242     17.8   
3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222     18.7   
4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222     18.7   
..       ...   ...    ...  ...    ...    ...   ...     ...  ..  ...      ...   
501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273     21.0   
502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273     21.0   
503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273     21.0   
504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273     21.0   
505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273     21.0   

         

In [ ]:
import pandas as pd
window_size = 3
data['smothingRM'] = data['rm'].rolling(window=window_size, min_periods=2).mean()
data['smothingRM']

0           NaN
1      6.498000
2      6.727000
3      6.868000
4      7.110000
         ...   
501    6.063000
502    6.246667
503    6.563000
504    6.630000
505    6.600000
Name: smothingRM, Length: 506, dtype: float64

In [ ]:
def normalizationMinMax(column):
    min= column.min()
    max = column.max()
    normalized_column = (column - min) / (max - min)
    return normalized_column

In [ ]:
data['taxNORMALIZED'] = normalizationMinMax(data['tax'])
data['taxNORMALIZED']

0      0.208015
1      0.104962
2      0.104962
3      0.066794
4      0.066794
         ...   
501    0.164122
502    0.164122
503    0.164122
504    0.164122
505    0.164122
Name: taxNORMALIZED, Length: 506, dtype: float64

In [ ]:

data['lstatNORMALIZED'] = normalizationMinMax(data['lstat'])
data['lstatNORMALIZED']

0      0.089680
1      0.204470
2      0.063466
3      0.033389
4      0.099338
         ...   
501    0.219095
502    0.202815
503    0.107892
504    0.131071
505    0.169702
Name: lstatNORMALIZED, Length: 506, dtype: float64

In [ ]:
X = data[['rm']]
y = data['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([23.76858826, 26.95061492, 19.73988199, 20.50282405, 22.66139185,
       22.49391676, 19.09789415, 21.51697875, 22.02870818, 20.14926553,
       19.4421485 , 19.93526959,  6.11857487, 22.0566207 , 17.18123481,
       29.016141  , 18.77224815,  9.47738079, 37.34337452, 21.61932463,
       22.13105407, 24.38266358, 17.22775567, 25.41542662, 19.95387793,
       14.56676262, 18.72572729, 26.67148977, 19.52588604, 17.98139357,
       15.91586749, 23.86162997, 32.11443012, 13.96199147, 19.92596542,
       17.19053899, 30.92349616, 18.67920643, 20.49351988, 22.17757493,
       14.22250827, 27.79729453, 38.82273779, 16.12055926, 23.77789243,
       17.22775567, 17.11610561, 23.77789243, 21.45184955, 25.24795153,
       19.98179045, 28.90449094, 21.21924526, 27.16461087, 33.57518505,
       22.80095442, 21.93566647, 27.90894459, 23.51737563, 16.22290515,
       26.98783161, 31.51896314, 25.03395559, 14.83658359, 25.28516822,
       13.79451638, 24.28031769, 22.7079127 , 25.43403496, 19.67

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse

42.74963952740483

In [ ]:
r2 = r2_score(y_test, y_pred)
r2

0.38952758463985193

Q1 part b

Part 2: Beautiful Soup [15 Marks]
You are provided with the URL of a website that lists the top-rated movies. Your task is to write
Python code using Beautiful Soup to scrape to perform following tasks.
URL: https://www.imdb.com/
● Scrape the movies released between 2013 and 2023.
● Write a Python script using BeautifulSoup to scrape the following information for each
movie:
⮚ Movie Title
⮚ Release Year
⮚ IMDb Rating
⮚ Director
⮚ Genre
● Store the scraped data in a structured format, such as a CSV file. ● Create functions for
each of the tasks listed below:
⮚ Average IMDb rating for the top-rated movies.
⮚ The most common genre among the top-rated movies.
⮚ Identify the director with the highest average IMDb rating.
⮚ Determine the year with the highest number of top-rated movies.

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
sources=requests.get('https://www.imdb.com/search/title/?release_date=2013-01-01,2023-09-24&start=201&ref_=adv_nxt')
soup=BeautifulSoup(sources.text,'html.parser')
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Released between 2013-01-01 and 2023-09-24
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?release_date=2013-01-01,2023-09-24&amp;start=201" rel="canonical"/>
<meta content="ht

In [ ]:
movies=soup.find(class_="lister-list").find_all(class_="lister-item mode-advanced")
len(movies)
for movie in movies:
  name=movie.find(class_='lister-item-content').a.text
  print(name)

熊蓋毒
Love Again
太空無垠
最後的王國
小丑
捍衛戰士：獨行俠
Miracle Workers
Scam 2003 - The Telgi Story
黑錢勝地
The Burial
敲敲門
異星入境
伯爵
仲夏魘
毒疫
駭客軍團
好球天團
殺手進城
紙房子
黑鳥
Poolman
No One Will Save You
恰吉
駭客任務：復活
勁爆女子監獄
水行俠
伊倫嘉：誘惑的藝術
限時追捕
極地追擊
西方極樂園
竊盜警長
Wolf
Heels
Champions
Baldur's Gate III
暗夜情報員
Priscilla
Annika
天能
漫才梅索太太
我的過錯
子彈列車
舞出新方向
喜幹會
我媽的希臘婚禮
杜巴利伯爵夫人
秘密入侵
瘋狂麥斯：憤怒道
The Chi
富家窮路


name of movies

In [ ]:
listt= []
for movie in movies:
    # Name
    name = movie.find(class_='lister-item-content').a.text

    # Year
    year = movie.find(class_='lister-item-year text-muted unbold').text

    # Rating
    rating = movie.find(class_='inline-block ratings-imdb-rating')
    ratingg = rating['data-value'] if rating else 'N/A'

    # Director
    director_element = movie.select_one('p:contains("Director:") a[href^="/name/"]')
    director_name = director_element.text if director_element else 'N/A'

    # Genre
    genre = movie.find(class_='genre').text

    # Append the movie details including the director's name to the list
    listt.append([name, year, ratingg, genre, director_name])


In [ ]:
listt

[['熊蓋毒', '(2023)', '5.9', '\nComedy, Thriller            ', 'Elizabeth Banks'],
 ['Love Again',
  '(I) (2023)',
  '5.8',
  '\nComedy, Drama, Romance            ',
  'Jim Strouse'],
 ['太空無垠', '(2015–2022)', '8.5', '\nDrama, Mystery, Sci-Fi            ', 'N/A'],
 ['最後的王國',
  '(2015–2022)',
  '8.5',
  '\nAction, Drama, History            ',
  'N/A'],
 ['小丑',
  '(I) (2019)',
  '8.4',
  '\nCrime, Drama, Thriller            ',
  'Todd Phillips'],
 ['捍衛戰士：獨行俠',
  '(2022)',
  '8.3',
  '\nAction, Drama            ',
  'Joseph Kosinski'],
 ['Miracle Workers',
  '(2019– )',
  '7.1',
  '\nComedy, Drama, Fantasy            ',
  'N/A'],
 ['Scam 2003 - The Telgi Story',
  '(2023)',
  '8.3',
  '\nBiography, Crime, Drama            ',
  'N/A'],
 ['黑錢勝地', '(2017–2022)', '8.5', '\nCrime, Drama, Thriller            ', 'N/A'],
 ['The Burial', '(II) (2023)', '8', '\nDrama            ', 'Maggie Betts'],
 ['敲敲門',
  '(2023)',
  '6.1',
  '\nHorror, Mystery, Thriller            ',
  'M. Night Shyamalan'],
 ['異星入

In [ ]:
import csv

csv_file_path = 'file.csv'

# Write the data to the CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(['Name', 'Year', 'Rating','Genre','Director'])

    # Write each row of movie data from the listt
    for movie_info in listt:
        writer.writerow(movie_info)


In [ ]:
iMbD=pd.read_csv("/content/file.csv")
iMbD

,Name,Year,Rating,Genre,Director
0,熊蓋毒,(2023),5.9,"\nComedy, Thriller",NaN
1,Love Again,(I) (2023),5.8,"\nComedy, Drama, Romance",NaN
2,太空無垠,(2015–2022),8.5,"\nDrama, Mystery, Sci-Fi",NaN
3,最後的王國,(2015–2022),8.5,"\nAction, Drama, History",NaN
4,小丑,(I) (2019),8.4,"\nCrime, Drama, Thriller",NaN
...,...,...,...,...,...
445,杜巴利伯爵夫人,(2023),6.8,"\nBiography, Drama, History",Maïwenn
446,秘密入侵,(2023),6.0,"\nAction, Adventure, Drama",NaN
447,瘋狂麥斯：憤怒道,(2015),8.1,"\nAction, Adventure, Sci-Fi",George Miller
448,The Chi,(2018– ),7.6,\nDrama,NaN


In [ ]:

def avgIMBD(df):
    # Remove rows with 'N/A' in the 'Rating' column
    skipNA = df[df['Rating'] != 'N/A']

    # Convert the 'Rating' column to float
    skipNA['Rating'] = skipNA['Rating'].astype(float)

    # Calculate the average IMDb rating
    avgRating = skipNA['Rating'].mean()

    return avgRating
#caliing funct
average_rating = avgIMBD(iMbD)
print("Average IMDb Rating:", average_rating)


Average IMDb Rating: 7.3919999999999995


In [ ]:
from collections import Counter

In [ ]:

def GENEREoccur(df):
    allGenres = []
    for genres in df['Genre']:
        if not pd.isna(genres):
            genreList = [genre.strip() for genre in genres.split(',')]
            allGenres.extend(genreList)


    genre_counts = Counter(allGenres)

    return genre_counts

genre_occurrences = GENEREoccur(iMbD)
print("Genre Occurrences:", genre_occurrences)


Genre Occurrences: Counter({'Drama': 136, 'Comedy': 64, 'Action': 56, 'Crime': 52, 'Thriller': 44, 'Mystery': 32, 'Sci-Fi': 28, 'Romance': 16, 'Fantasy': 16, 'Biography': 16, 'Horror': 16, 'Adventure': 16, 'History': 12, 'Sport': 12, 'Elizabeth Banks': 5, 'Jim Strouse': 5, 'Todd Phillips': 5, 'Joseph Kosinski': 5, 'Maggie Betts': 5, 'M. Night Shyamalan': 5, 'Denis Villeneuve': 5, 'Pablo Larraín': 5, 'Ari Aster': 5, 'Chris Pine': 5, 'Brian Duffield': 5, 'Lana Wachowski': 5, 'James Wan': 5, 'Bille August': 5, 'Damián Szifron': 5, 'Taylor Sheridan': 5, 'Bobby Farrelly': 5, 'Swen Vincke': 5, 'Sofia Coppola': 5, 'Christopher Nolan': 5, 'Domingo González': 5, 'David Leitch': 5, 'Adele Lim': 5, 'Kirk Jones': 5, 'Maïwenn': 5, 'George Miller': 5, 'Music': 4, 'Steven Strait': 4, 'Alexander Dreymon': 4, 'Daniel Radcliffe': 4, 'Gagandev Riar': 4, 'Jason Bateman': 4, 'Michael Keaton': 4, 'Rami Malek': 4, 'Dwayne Johnson': 4, 'Bill Hader': 4, 'Úrsula Corberó': 4, 'Taron Egerton': 4, 'Zackary Arthur'

In [ ]:

most_common_genre, most_common_count = genre_occurrences.most_common(1)[0]
most_common_genre

'Drama'

In [ ]:
print('Year with the Highest Number of Top-rated Movies:', most_common_genre, '(', most_common_count, 'movies)')

Year with the Highest Number of Top-rated Movies: Drama ( 136 movies)


Determine the year with the highest number of top-rated movies.

In [ ]:


def highTopMovies(df):
    yearCounts = df['Year'].value_counts()
    yearMAX = yearCounts.idxmax()
    countyearMAX = yearCounts.max()

    result = 'Year with the Highest Number of Top-rated Movies: ' + str(yearMAX) + ' (' + str(countyearMAX) + ' movies)'

    return result


result = highTopMovies(iMbD)
print(result)


Year with the Highest Number of Top-rated Movies: (2023) (126 movies)


Identify the director with the highest average IMDb rating.

In [ ]:
def directHighRating(df):
    df = df[df['Rating'] != 'N/A']
    df['Rating'] = df['Rating'].astype(float)
    director_avg_ratings = df.groupby('Director')['Rating'].mean()
    highest_rated_director = director_avg_ratings.idxmax()
    highest_avg_rating = director_avg_ratings.max()
    result =( "Director with the Highest Average IMDb Rating:",highest_rated_director, highest_avg_rating)
    return result


result = directHighRating(iMbD)
print(result)


('Director with the Highest Average IMDb Rating:', 'Swen Vincke', 9.8)
